In [ ]:
#Q1
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt 
from keras.datasets import fashion_mnist
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
names = ["T-shirt/top","Trouser/pants","Pullover shirt","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]
dummy = []
for i in range (10):
  for j in range(len(Y_train)):
    if int(Y_train[j]) is i:
      plt.imshow(X_train[j], cmap="gray")
      dummy.append(X_train[j])
      plt.show()
      break 
wandb.log(({"Q1":[wandb.Image(i,caption=j) for i,j in zip(dummy,names)]}))

In [ ]:
import numpy as np
import random
import cv2
random.seed(1)

In [ ]:
def sgd(run,n_epoch,batch_size=100,eta = 0.01):
  global W,b,arguments,n_input,n_points ,n_output, n_hidden_layers,n_hidden_neurons , X,Y,X_t,Y_t,y_o,y_t
  
  d_w , d_b = mul(W , 0) , mul(b,0)
  train_loss , train_acc = [] , []
  val_loss , val_acc = [] , []

  for i in range(n_epoch):
    for j in range(n_points):
      t1 , t2 , _ = gradient(X[j],Y[j],W,b)
      d_w = add(d_w , t1)
      d_b = add(d_b , t2)
      

      if(j%batch_size == 0):
        # print("dw",d_w,"db",d_b)
        W = add(W , d_w , 1, -1*eta)
        b = add(b , d_b , 1, -1*eta)
        d_w , d_b = mul(W , 0) , mul(b,0)
        

    # train acc and loss
    t4 , t5 = accuracy_and_loss(X,Y,y_o)
    # print(W,b)
    train_loss.append(t5)
    train_acc.append(t4)

    # validate
    va , vl = accuracy_and_loss(X_t,Y_t,y_t)
    val_loss.append(vl)
    val_acc.append(va)
    print("epoch",i,"train acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)
    # wandb.log({'train_acc' : t4 , 'train_loss' : t5},step = i) 
    vl = float (vl)
    t5 = float (t5)
    run.log({'t_acc' : t4 ,'t_loss' : t5,'v_loss' : vl,'v_acc':va})

  return W , b , train_loss , train_acc , val_loss, val_acc

def momentum_gradient_descent(run,n_epoch,batch_size=100,eta = 0.001,gamma = 0.5):
  
  global W,b,arguments,n_input,n_points ,n_output, n_hidden_layers,n_hidden_neurons , X,Y,X_t,Y_t,y_o,y_t
  
  d_w , d_b = mul(W , 0) , mul(b,0)
  train_loss , train_acc = [] , []
  val_loss , val_acc = [] , []
  p_w , p_b =mul(W , 0) , mul(b,0)
  v_w , v_b =mul(W , 0) , mul(b,0)

  for i in range(n_epoch):
    for j in range(n_points):
      t1 , t2 , _ = gradient(X[j],Y[j],W,b)
      d_w = add(d_w , t1)
      d_b = add(d_b , t2)   
    
      if(j%batch_size == 0):
        v_w , v_b = add(p_w , d_w , gamma , eta) , add(p_b , d_b , gamma , eta)
        W = add(W , v_w , 1, -1)
        b = add(b , v_b , 1 , -1)
        p_w , p_b = v_w , v_b
        # print("dw",d_w,"db",d_b)
        d_w , d_b = mul(W , 0) , mul(b,0)
        
    # train acc and loss
    t4 , t5 = accuracy_and_loss(X,Y,y_o)
    train_loss.append(t5)
    train_acc.append(t4)

    # validate
    va , vl = accuracy_and_loss(X_t,Y_t,y_t)
    val_loss.append(vl)
    val_acc.append(va)
    print("epoch",i,"trian acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)
    # wandb.log({'train_acc' : train_acc[-1] , 'train_loss' : train_loss[-1] , 'val_acc' : val_acc[-1], 'val_loss': val_loss[-1]})
    # wandb.log({'train_acc' : t4 , 'train_loss' : t5},step = i)
    vl = float (vl)
    t5 = float (t5)
    run.log({'t_acc' : t4 ,'t_loss' : t5,'v_loss' : vl})
  return W , b , train_loss , train_acc , val_loss, val_acc

In [ ]:
def onehot_encoding(a,n_class):   
  temp = []
  for i in a:
    t1 = np.zeros(n_class)
    t1[i] = 1
    temp.append(t1)
  return temp

def squr(d):    
  temp = {}
  for i in d.keys():
    temp[i] = d[i]**2
  return temp

def mul(d1 , m1 = 1):   
  temp ={}
  for i in d1.keys():
    temp[i] = m1 * d1[i]
  return temp

# dictionary add key wise
def add(d1,d2,m1=1,m2=1):
  temp ={}
  # print(d1,d2,"dd")
  if (m2==0):
    return d1
  for i in d1.keys():
    temp[i] = m1 * d1[i] + m2 * d2[i]
  return temp

In [ ]:
def nesterov_gradient_descent(run,n_epoch,batch_size=100,eta = 0.001,gamma = 0.9):

  global W,b,arguments,n_input,n_points ,n_output, n_hidden_layers,n_hidden_neurons , X,Y,X_t,Y_t,y_o,y_t
  p_v_w , p_v_b = mul(W , 0) , mul(b,0)
  v_w , v_b = mul(W , 0) , mul(b,0)
  d_w , d_b = mul(W , 0) , mul(b,0)
  train_loss , train_acc = [] , []
  val_loss , val_acc = [] , []

  for i in range(n_epoch):
    for j in range(n_points):
      t1 , t2 , _ = gradient(X[j],Y[j],add(W,v_w,1,-gamma),add(b,v_b,1,-gamma))
      d_w = add(d_w , t1)
      d_b = add(d_b , t2)
      
      if(j%batch_size == 0):
        v_w , v_b = add(p_v_w , d_w , gamma , eta) , add(p_v_b , d_b , gamma , eta)
        W = add(W , v_w , 1, -1)
        b = add(b , v_b , 1 , -1)
        d_w , d_b = mul(W , 0) , mul(b,0)
        p_v_w , p_v_b = v_w , v_b

    # train acc and loss
    t4 , t5 = accuracy_and_loss(X,Y,y_o)
    train_loss.append(t5)
    train_acc.append(t4)

    # validate
    va , vl = accuracy_and_loss(X_t,Y_t,y_t)
    val_loss.append(vl)
    val_acc.append(va)
    print("epoch",i,"trian acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)
    # wandb.log({'train_acc' : train_acc[-1] , 'train_loss' : train_loss[-1] , 'val_acc' : val_acc[-1], 'val_loss': val_loss[-1]})
    # wandb.log({'train_acc' : t4 , 'train_loss' : t5},step = i)
    vl = float (vl)
    t5 = float (t5)
    run.log({'t_acc' : t4 ,'t_loss' : t5,'v_loss' : vl,'v_acc':va})

  return W , b , train_loss , train_acc , val_loss, val_acc

def adagrad(run,n_epoch,batch_size=100,eta = 0.001,eps = 1e-8):
  
  global W,b,arguments,n_input,n_points ,n_output, n_hidden_layers,n_hidden_neurons , X,Y,X_t,Y_t,y_o,y_t
  v_w , v_b = mul(W , 0) , mul(b,0)
  d_w , d_b = mul(W , 0) , mul(b,0)
  train_loss , train_acc = [] , []
  val_loss , val_acc = [] , []

  for i in range(n_epoch):
    for j in range(n_points):
      t1 , t2 , _ = gradient(X[j],Y[j],W,b)
      d_w = add(d_w , t1)
      d_b = add(d_b , t2)
      
      if(j%batch_size == 0):
        v_w , v_b = add(v_w, squr(d_w)) , add(v_b , squr(d_b))
        W = add(W , adarate(d_w,v_w,eta,eps) , 1, -1)
        b = add(b , adarate(d_b,v_b,eta,eps) , 1 , -1)
        d_w , d_b = mul(W , 0) , mul(b,0)    

    # train acc and loss
    t4 , t5 = accuracy_and_loss(X,Y,y_o)
    train_loss.append(t5)
    train_acc.append(t4)

    # validate
    va , vl = accuracy_and_loss(X_t,Y_t,y_t)
    val_loss.append(vl)
    val_acc.append(va)
    print("epoch",i,"trian acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)
    # wandb.log({'train_acc' : train_acc[-1] , 'train_loss' : train_loss[-1] , 'val_acc' : val_acc[-1], 'val_loss': val_loss[-1]})
    # wandb.log({'train_acc' : t4 , 'train_loss' : t5},step = i)
    vl = float (vl)
    t5 = float (t5)
    run.log({'t_acc' : t4 ,'t_loss' : t5,'v_loss' : vl,'v_acc':va})
  return W , b , train_loss , train_acc , val_loss, val_acc 

In [ ]:
# y_o original output y_p predicted output
def cross_entropy(y_o,y_p):
  for i,j in zip(y_o,y_p):
    if (i==1 and j >= 0):
      return -1*np.log(1e-15+j,dtype=np.float128) + regularize_loss()
  return 1e+3 

#  gradient for the oputput layer when cross entropy is used
def grad_cross_entropy(y_o,y_p):#PMS
  return -(y_o - y_p) 

def regularize_loss():
  global W,lambda1
  temp = squr(W)
  # print(temp)
  total = 0
  for i in temp.keys():
    total = total + np.sum(temp[i])

  return lambda1 * total
# regularize_loss()

def squared_error(y_o,y_p):#PMS
  return  np.sum((np.array(y_o)-np.array(y_p))**2) + regularize_loss()

def grad_squared_error(y_o,y_p):#PMS
  # print(y_o,y_p)
  y_o = list(y_o)
  y_p = list(y_p)
  temp =[]
  ind = y_o.index(max(y_o))
  for i in range(len(y_o)):
    temp.append(-1* 2 * y_p[ind] * (y_o[i]-y_p[ind]) * (y_o[i]-y_p[i]))

  return np.array(temp)
# grad_squared_error([1,0,0],[0.1,0.2,0.3])

In [ ]:
# a is list
def softmax(a):
  p = []
  for i in a:
    if(i <= 1e+4 ):
      if(i >= -1e+4):
        p.append(np.exp(i,dtype=np.float128))
      else:
        p.append(0)
    else:
      return a/np.sum(a)
  if(np.sum(p) == 0):
    return p
  return p/np.sum(p)

def gradient(x,y,W,b):  
  global arguments,n_input,n_points ,n_output, n_hidden_layers,n_hidden_neurons
  # forward pass
  a , h = forward_pass(x,W,b)
  # print(a,h)
  # output activation
  p = globals()[arguments[1]](a[n_hidden_layers])

  # loss
  l = globals()[arguments[2]](y,p)

  # backward pass
  d_w ,d_b = backward_pass(x,y,p,a,h,W,b)
  # print(d_w , d_b)

  return d_w , d_b , l

In [ ]:
def forward_pass(X,W,b):  
  global arguments,n_input,n_output, n_hidden_layers,n_hidden_neurons
  # h does not contain X
  h={}
  a={}
  # input 
  a[0] = np.array(X @ W[0] + b[0],dtype=np.float32)
  # print(a[0])
  
  
  h[0] = list(map(lambda x : globals()[arguments[0]](x),a[0]))
  # print(h[0])

  # hidden
  for i in range(1,n_hidden_layers):
    a[i] = np.array(h[i-1] @ W[i] + b[i])
    h[i] = list(map(lambda x : globals()[arguments[0]](x),a[i]))

  # output
  a[n_hidden_layers] = np.array(h[n_hidden_layers-1] @ W[n_hidden_layers] + b[n_hidden_layers])

  # print("a",a,"h")

  return a , h


# forward_pass(np.random.rand(n_input),n_input,n_output=4,n_hidden_layers=2,n_hidden_neurons=[2,3])

In [ ]:
def backward_pass(x,y,p,a,h,W,b):
  global arguments,n_input,n_output, n_hidden_layers,n_hidden_neurons,lambda1
  d_al = None
  d_h = None
  total_layers = n_hidden_layers + 2
  inner_activation = arguments[0]
  output_activation = arguments[2]

  # gradient for output w.r.t a_l
  # print("o",y)
  d_al = globals()["grad_"+output_activation](y,p)

  d_w ={}
  d_b ={}
  #  for all hidden layers
  for i in range(n_hidden_layers,0,-1):
    d_w[i] = np.array([np.dot(d_al,h[i-1][j]) for j in range(n_hidden_neurons[i-1])])
    
    d_b[i] = np.array(d_al)

    d_h = np.array((np.matrix(W[i]) @ np.matrix(d_al).T).T)[0]
    # print("d_h",d_h)

    d_al = [d_h[j] * globals()["grad_"+inner_activation](a[i-1][j]) for j in range(n_hidden_neurons[i-1])]

  # for input layer
  d_w[0] = np.array([np.dot(d_al,x[j]) for j in range(n_input)])
  d_b[0] =  np.array(d_al)

  return add(d_w,W,1,lambda1) , d_b

In [ ]:
def create_network_random():  #PMS
  # input + output = 2
  total_layers = 2 + n_hidden_layers 
  W = {}
  b = {}
  #  for xavier t1 = 1/np.squrt((ni no of input neurons +no of output neurons))
  t1 = 1
  # initialization for W0 and b0 i.e. input layer
  W[0] = np.random.randn(n_input,n_hidden_neurons[0])/t1
  b[0] = np.random.randn(n_hidden_neurons[0])/t1
  # print(type(W[0]))

  # hidden layer
  for i in range(1,n_hidden_layers):
    W[i] = np.random.randn(n_hidden_neurons[i-1],n_hidden_neurons[i])/t1
    b[i] = np.random.randn(n_hidden_neurons[i])/t1
    

  # output layer
  W[total_layers-2] = np.random.randn(n_hidden_neurons[-1],n_output)/t1
  b[total_layers-2] = np.random.randn(n_output)/t1

  # print("W",len(W))
  # for i in W.values():
  #   print(np.shape(i)) 
  # print("b",len(b))
  # for i in b.values():
  #   print(len(i))

  return W,b
# create_network()
def create_network_xavier():  #PMS
  # input + output = 2
  total_layers = 2 + n_hidden_layers 
  W = {}
  b = {}
  #  for xavier t1 = 1/np.squrt((ni no of input neurons +no of output neurons))
  t1 = 1
  # initialization for W0 and b0 i.e. input layer
  W[0] = np.random.rand(n_input,n_hidden_neurons[0]) * np.sqrt((n_input+n_hidden_neurons[0]))
  b[0] = np.random.rand(n_hidden_neurons[0]) * 0
  # print(type(W[0])

  # hidden layer
  for i in range(1,n_hidden_layers):
    W[i] = np.random.rand(n_hidden_neurons[i-1],n_hidden_neurons[i]) * np.sqrt((n_hidden_neurons[i-1] + n_hidden_neurons[i]))
    b[i] = np.random.rand(n_hidden_neurons[i]) * 0
    

  # output layer
  W[total_layers-2] = np.random.rand(n_hidden_neurons[-1],n_output) * np.sqrt((n_hidden_neurons[-1] + n_output))
  b[total_layers-2] = np.random.rand(n_output) * 0

  # print("W",len(W))
  # for i in W.values():
  #   print(np.shape(i)) 
  # print("b",len(b))
  # for i in b.values():
  #   print(len(i))

  return W,b
# create_network_xavier()

In [ ]:
def accuracy(y_o, y_p):
  sum = 0
  for i,j in zip(y_o , y_p):
    if(i == j):
      sum = sum + 1
  
  return sum/len(y_o)
def predict_and_loss(X,Y,n):
  global W,b,arguments,n_input,n_output, n_hidden_layers,n_hidden_neurons

  loss = 0
  predicted_class = []
  for i in range(n):
    # forward pass
    a , h = forward_pass(X[i],W,b)
    # if(i==0):
    #   print("a",a,"h",h)
    # output activation
    p = globals()[arguments[1]](a[n_hidden_layers])
    p = list(p)
    # print(p)
    predicted_class.append(p.index(max(p)))

    #  loss
    ll = globals()[arguments[2]](Y[i],p)
    if(np.isnan(ll) or np.isinf(ll)):
      loss = loss + 1e+100
    else:
      loss = loss + ll

  return predicted_class , loss/n


def accuracy_and_loss(X,Y,y):
  n_points , _ = np.shape(X)
  # print(len(Y))
  p , l = predict_and_loss(X,Y,n_points)
  # print(p)
  acc  = accuracy(y, p)

  return acc , l

In [ ]:

# n single vale
def sigmoid(n):   
  if(n >= 1e+2 ):
    return 1
  elif (n <= 1e-2):
    return 1e-3
  return 1/(1 + np.exp(-n,dtype= np.float128))
# print(sigmoid(3))
# s = [1,2,3]
# t={}
# t[0]=list(map(lambda x : sigmoid(x),s))
# print(t)
# single value n 
def grad_sigmoid(n):
  temp = sigmoid(n)
  return temp * (1 - temp)
def Relu(n):  
  if n <= 0:
    return 0
  else:
    return n
def grad_Relu(n):
  if n <= 0:
    return 0
  else:
    return 1
def tanh(n):
  if ( n >= 1e+2):
    return 1
  elif (n <= -1e+2):
    return -1
  else:
    return (np.exp(n,dtype=np.float128) - np.exp(-1*n)) / (np.exp(n) + np.exp(-1*n))
  return 0
def grad_tanh(n):
  return 1-np.power(tanh(n),2)

In [ ]:
def rmsprop(run,n_epoch,batch_size=100,eta = 0.001,eps = 1e-8,beta1=0.9):
  
  global W,b,arguments,n_input,n_points ,n_output, n_hidden_layers,n_hidden_neurons , X,Y,X_t,Y_t,y_o,y_t
  v_w , v_b = mul(W , 0) , mul(b,0)
  d_w , d_b = mul(W , 0) , mul(b,0)
  train_loss , train_acc = [] , []
  val_loss , val_acc = [] , []


  for i in range(n_epoch):
    for j in range(n_points):
      t1 , t2 , _ = gradient(X[j],Y[j],W,b)
      d_w = add(d_w , t1)
      d_b = add(d_b , t2)
      
      if(j%batch_size == 0):
        v_w , v_b = add(v_w, squr(d_w),beta1,(1-beta1)) , add(v_b , squr(d_b),beta1,(1-beta1))
        W = add(W , adarate(d_w,v_w,eta,eps) , 1, -1)
        b = add(b , adarate(d_b,v_b,eta,eps) , 1 , -1)
        d_w , d_b = mul(W , 0) , mul(b,0)
        
    # train acc and loss
    t4 , t5 = accuracy_and_loss(X,Y,y_o)
    train_loss.append(t5)
    train_acc.append(t4)

    # validate
    va , vl = accuracy_and_loss(X_t,Y_t,y_t)
    val_loss.append(vl)
    val_acc.append(va)
    print("epoch",i,"trian acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)
    # wandb.log({'train_acc' : train_acc[-1] , 'train_loss' : train_loss[-1] , 'val_acc' : val_acc[-1], 'val_loss': val_loss[-1]})
    # wandb.log({'train_acc' : t4 , 'train_loss' : t5},step = i)
    vl = float (vl)
    t5 = float (t5)
    run.log({'t_acc' : t4 ,'t_loss' : t5,'v_loss' : vl,'v_acc':va})
  return train_loss , train_acc , val_loss, val_acc 
def adam(run,n_epoch,batch_size=100,eta = 0.001,eps = 1e-8,beta1 = 0.9,beta2 =0.999):
  
  global W,b,arguments,n_input,n_points ,n_output, n_hidden_layers,n_hidden_neurons , X,Y,X_t,Y_t,y_o,y_t
  v_w , v_b = mul(W , 0) , mul(b,0)
  m_w , m_b = mul(W , 0) , mul(b,0)
  d_w , d_b = mul(W , 0) , mul(b,0)
  train_loss , train_acc = [] , []
  val_loss , val_acc = [] , []
  time_stamp = 0
  for i in range(n_epoch):
    for j in range(n_points):
      t1 , t2 , _ = gradient(X[j],Y[j],W,b)
      d_w = add(d_w , t1)
      d_b = add(d_b , t2)
      
      if(j%batch_size == 0):
        
        m_w , m_b = add(m_w,d_w,beta1 ,(1-beta1)) , add(m_b,d_b,beta1 ,(1-beta1)) 
        v_w , v_b = add(v_w, squr(d_w),beta2,(1-beta2)) , add(v_b , squr(d_b),beta2,(1-beta2))
    
        m_w_hat , m_b_hat = mul(m_w , 1/(1-beta1**(time_stamp+1))) ,  mul(m_b , 1/(1-beta1**(time_stamp+1)))
        v_w_hat , v_b_hat = mul(v_w , 1/(1-beta2**(time_stamp+1))) ,  mul(v_b , 1/(1-beta2**(time_stamp+1)))  

        W = add(W , adarate(m_w_hat,v_w_hat,eta,eps) , 1, -1)
        b = add(b , adarate(m_b_hat,v_b_hat,eta,eps) , 1 , -1)
        d_w , d_b = mul(W , 0) , mul(b,0)
        time_stamp = time_stamp+1
        
    # train acc and loss
    t4 , t5 = accuracy_and_loss(X,Y,y_o)
    train_loss.append(t5)
    train_acc.append(t4)

    # validate
    va , vl = accuracy_and_loss(X_t,Y_t,y_t)
    val_loss.append(vl)
    val_acc.append(va)
    print("epoch",i,"trian acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)
    # wandb.log({'train_acc' : train_acc[-1] , 'train_loss' : train_loss[-1] , 'val_acc' : val_acc[-1], 'val_loss': val_loss[-1]})
    # wandb.log({'train_acc' : t4 , 'train_loss' : t5},step = i)
    vl = float (vl)
    t5 = float (t5)
    run.log({'t_acc' : t4 ,'t_loss' : t5,'v_loss' : vl,'v_acc':va})
  return train_loss , train_acc , val_loss, val_acc   

        
# np.seterr(divide='ignore', invalid='ignore')
def nadam(run,n_epoch,batch_size=100,eta = 0.001,eps = 1e-8,beta1 = 0.9,beta2 =0.999):

  global W,b,arguments,n_input,n_points ,n_output, n_hidden_layers,n_hidden_neurons , X,Y,X_t,Y_t,y_o,y_t
  v_w , v_b = mul(W , 0) , mul(b,0)
  m_w , m_b = mul(W , 0) , mul(b,0)
  d_w , d_b = mul(W , 0) , mul(b,0)
  train_loss , train_acc = [] , []
  val_loss , val_acc = [] , []
  time_stamp=0
  for i in range(n_epoch):
    for j in range(n_points):
      t1 , t2 , _ = gradient(X[j],Y[j],add(W,m_w,1,-beta1),add(b,m_b,1,-beta1))
      d_w = add(d_w , t1)
      d_b = add(d_b , t2)
      
      if(j%batch_size == 0):
        m_w , m_b = add(m_w,d_w,beta1 ,(1-beta1)) , add(m_b,d_b,beta1 ,(1-beta1)) 
        v_w , v_b = add(v_w, squr(d_w),beta2,(1-beta2)) , add(v_b , squr(d_b),beta2,(1-beta2))
    
        m_w_hat , m_b_hat = mul(m_w , 1/(1-beta1**(time_stamp+1))) ,  mul(m_b , 1/(1-beta1**(time_stamp+1)))
        v_w_hat , v_b_hat = mul(v_w , 1/(1-beta2**(time_stamp+1))) ,  mul(v_b , 1/(1-beta2**(time_stamp+1)))  

        W = add(W , adarate(add(m_w_hat,d_w,beta1,(1-beta1)/(1-beta1**(time_stamp+1))),v_w_hat,eta,eps) , 1, -1)
        b = add(b , adarate(add(m_b_hat,d_b,beta1,(1-beta1)/(1-beta1**(time_stamp+1))),v_b_hat,eta,eps) , 1 , -1)
        d_w , d_b = mul(W , 0) , mul(b,0)
        time_stamp = time_stamp +1
        
    # train acc and loss
    t4 , t5 = accuracy_and_loss(X,Y,y_o)
    train_loss.append(t5)
    train_acc.append(t4)

    # validate
    va , vl = accuracy_and_loss(X_t,Y_t,y_t)
    val_loss.append(vl)
    val_acc.append(va)
    print("epoch",i,"trian acc", t4 , "train loss" ,t5 , "validation acc" , va , "validation loss" ,vl)
    vl = float (vl)
    t5 = float (t5)
    run.log({'t_acc' : t4 ,'t_loss' : t5,'v_loss' : vl,'v_acc':va})
    # wandb.log({'train_acc' : t4 , 'train_loss' : t5},step = i)

  return train_loss , train_acc , val_loss, val_acc   

        
# np.seterr(divide='ignore', invalid='ignore')